<a href="https://colab.research.google.com/github/adityajivoji/mnist_classifier/blob/main/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# Importing Libraries
import torch
import numpy as np
import torchvision
from torch.utils.data import DataLoader
from torch import nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch.nn import functional as F

In [91]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # checking device type

In [92]:
# Convolutional Neural Network
class cnn(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 5, padding= 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.FC1 = nn.Linear(16 * 7 * 7, 300) # image of size 28 * 28 are reduced to 7 * 7 by two pool operations of size (2,2)
        self.FC2 = nn.Linear(300, 112)
        self.FC3 = nn.Linear(112, 120)
        self.FC4 = nn.Linear(120, 10) # Final Layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.FC1(x))
        x = F.relu(self.FC2(x))
        x = F.relu(self.FC3(x))
        x = self.FC4(x)
        return x

In [93]:
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 3

In [94]:
train_dataset = datasets.MNIST(root = '.', train= True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '.', train= False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [95]:
model = cnn(in_channels, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [96]:
for epoch in range(num_epoch): # number of iterations
    for _, (data, targets) in enumerate(train_loader , 0): # one min-batch goes through each iteration of size 64
        # Converting to acceptable format
        data = data.to(device=device)
        targets = targets.to(device=device)
        output = model(data)
        loss = criterion(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [97]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
    model.train()
    return num_correct/num_samples * 100


In [98]:
training_accuracy = check_accuracy(train_loader, model)
testing_accuracy = check_accuracy(test_loader, model)
print(f"Training Accuracy: {training_accuracy}")
print(f"Testing Accuracy: {testing_accuracy}")


Training Accuracy: 99.1116714477539
Testing Accuracy: 98.8699951171875
